In [28]:
import argparse
import os
import model_utils
import pickle
import torch
import numpy as np
import scipy

from torch.autograd import Variable

import pdb

parent_dir = '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1'
gpu_ids = [0, 1]
batch_size = 100

# parser = argparse.ArgumentParser()
# parser.add_argument('--parent_dir', help='save dir')
# parser.add_argument('--gpu_ids', nargs='+', type=int, default=0, help='gpu id')
# parser.add_argument('--batch_size', type=int, default=400, help='batch_size')
# parser.add_argument('--overwrite', type=bool, default=False, help='overwrite existing results')
# args = parser.parse_args()

# model_dir = args.parent_dir + os.sep + 'struct_model' 


figure_dir = parent_dir + os.sep + 'figures'

if not os.path.exists(figure_dir):
    os.makedirs(figure_dir)



torch.manual_seed(2)
torch.cuda.manual_seed(2)
np.random.seed(2)




model_dir = parent_dir + os.sep + 'ref_model'
opt_ref = pickle.load(open( '{0}/opt.pkl'.format(model_dir), "rb" ))
print(opt_ref)
opt_ref.gpu_ids = gpu_ids

opt_ref.save_dir = model_dir
models, _, _, logger_ref, _ = model_utils.load_model(opt_ref.model_name, opt_ref)
enc_ref = models['enc']
dec_ref = models['dec']

enc_ref.train(False)
dec_ref.train(False)
models = None

dp_ref = model_utils.load_data_provider(opt_ref.data_save_path, opt_ref.imdir, opt_ref.dataProvider)
dp_ref.opts['channelInds'] = opt_ref.channelInds

model_dir = parent_dir + os.sep + 'struct_model'
opt = pickle.load(open( '{0}/opt.pkl'.format(model_dir), "rb" ))
print(opt)
opt.gpu_ids = gpu_ids

opt.save_dir = model_dir
models, _, _, logger, _ = model_utils.load_model(opt.model_name, opt)
enc = models['enc']
dec = models['dec']

enc.train(False)
dec.train(False)
models = None

dp = model_utils.load_data_provider(opt.data_save_path, opt.imdir, opt.dataProvider)


print('done loading models')

Namespace(Diters=5, DitersAlt=100, batch_size=30, channelInds=[0, 2], channels_pt1=[0, 2], channels_pt2=[0, 1, 2], critRecon='BCELoss', dataProvider='DataProvider3Dh5', data_save_path='/root//results/integrated_cell/test_aaegan/dgx_test_v4_1/data.pyt', decDRatio=1e-05, dtype='float', encDRatio=0.0001, gpu_ids=[0, 1], imdir='/root/data/ipp/ipp_17_10_25', latentDistribution='gaussian', lrDec=0.0002, lrDecD=0.0002, lrEnc=0.0002, lrEncD=0.01, model_name='aaegan3Dv6-exp', myseed=0, nClasses=0, nRef=0, nch=2, ndat=19755, nepochs=150, nepochs_pt2=150, nlatentdim=128, noise=0.01, optimizer='adam', saveProgressIter=1, saveStateIter=1, save_dir='/root//results/integrated_cell/test_aaegan/dgx_test_v4_1/ref_model', save_parent='/root//results/integrated_cell/test_aaegan/dgx_test_v4_1', train_module='aaegan_trainv6')
Loading from /root/results/integrated_cell/test_aaegan/dgx_test_v4_1/ref_model
Namespace(Diters=5, DitersAlt=100, batch_size=30, channelInds=[0, 1, 2], channels_pt1=[0, 2], channels_pt

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from imgToProjection import imgtoprojection
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
    
def tensor2img(imgs, colors=None):

    
    if colors is None:
            colormap = 'hsv'
            colors = plt.get_cmap(colormap)(np.linspace(0, 1, imgs.shape[1]+1))
    
    imgs = imgs.numpy()
    im_out = list()
    
    #for every image
    for i in range(0, imgs.shape[0]):
        img = imgs[i]
        
        #contrast adjust each channel
        for j in range(0, img.shape[0]):
            maxval = np.max(img[j])
            if maxval > 0:
                img[j] = img[j]/maxval
            
        
        img = imgtoprojection(np.swapaxes(img, 1, 3), proj_all=True, colors = colors, global_adjust=True)
        img = np.swapaxes(img, 0, 2)
        
        img = np.flip(np.flip(img,0), 1)
        
        #very light border between each projection
        img[:,-2:,:] = .25
        img[-2:,:,:] = .25
        
        im_out.append(img)

    img = np.concatenate(im_out, 1)

    return img

def print_fig1(dp, save_path, save_path_struct, n_per_row = 6, colors=[[1,0,1], [1,1,0], [0,1,1]]):
    #this figure is #classes by n_per_row max project image of cells. 

    ntrain = dp.get_n_dat('train')
    
    train_classes = dp.get_classes(np.arange(0, ntrain), 'train').numpy()
    class_names = dp.label_names
    
    image_list = list()
    image_list_struct = list()
    
    for i in range(0, len(class_names)):
        inds = np.where(train_classes == i)[0]
        np.random.shuffle(inds)
        
        #make the CMY image
        images = dp.get_images(inds[0:n_per_row], 'train')
        images_flat = tensor2img(images, colors)
        image_list.append(images_flat)
        
        #make the struct-only Yellow image
        images[:, [0,2], :, :,:] = images[:, [0,2], :, :,:].fill_(0) #not sure why i have to do it this way, but it doesn't work without the assigment
        
        images_flat_struct = tensor2img(images, colors)
        image_list_struct.append(images_flat_struct)
        
        
    images = np.concatenate(image_list, axis=0)
    scipy.misc.imsave(save_path, images)
    
    images_struct = np.concatenate(image_list_struct, axis=0)
    scipy.misc.imsave(save_path_struct, images_struct)
                                   
def print_fig2(enc, dec, dp, save_path, save_path_struct, n_per_row = 6, colors=[[1,0,1], [1,1,0], [0,1,1]], required_test_inds=None):
    ntrain = dp.get_n_dat('train')
    train_classes = dp.get_classes(np.arange(0, ntrain), 'train').numpy()
    
    class_names = dp.label_names
    
    nclasses = len(class_names)
    train_inds = np.int_(np.zeros(nclasses))
    for i in range(0, len(class_names)):
        images_w = list()
        class_inds = np.where(train_classes == i)[0]
        np.random.shuffle(class_inds)
        train_inds[i] = class_inds[0]
        
    train_images = dp.get_images(train_inds, 'train')
    train_images_struct = train_images.clone()
    train_images_struct[:, [0,2], :, :,:] = train_images_struct[:, [0,2], :, :,:].fill_(0)
    
    
    n_per_row_tmp = n_per_row - len(required_test_inds)
    
    ntest = dp.get_n_dat('test')
    test_inds= np.arange(0, ntest)
    np.random.shuffle(test_inds)
    test_inds = test_inds[np.arange(0, n_per_row_tmp-1)]
    
    test_inds = np.concatenate([test_inds, required_test_inds])
    
    test_images = dp.get_images(test_inds, 'test')
    
    images_h = list()
    images_h_struct = list()   
    
    for i in range(0, len(class_names)):
        images_w = list()
        images_w_struct = list()
        
        train_inds = np.where(train_classes == i)[0]
        np.random.shuffle(train_inds)
        
        for j in range(0, n_per_row-1):
            image = Variable(torch.unsqueeze(test_images[j], 0))
            z = enc(image)
            
            z[0].data.fill_(-100)
            z[0].data[0,i] = 0

            z[-1].data.fill_(0)
    
            image = dec(z)
        
            image = image.data.cpu()
        
            image[:, [0,2], :, :,:] = test_images[j][[0,2],:,:,:]
        
            images_w.append(tensor2img(image, colors))
            
            image[:, [0,2], :, :,:] = image[:, [0,2], :, :,:].fill_(0)
            images_w_struct.append(tensor2img(image, colors))
        
        #CMY version
        images_w = np.concatenate(images_w, axis=1)
        images_w[:,-2:,:] = 0.5
        image_flat = tensor2img(torch.unsqueeze(train_images[i], 0), colors)
        images_w = np.concatenate([images_w, image_flat], axis=1)        
        images_h.append(images_w)
        
        #Struct-only Yellow version
        images_w_struct = np.concatenate(images_w_struct, axis=1)
        images_w_struct[:,-2:,:] = 0.5
        image_flat = tensor2img(torch.unsqueeze(train_images_struct[i], 0), colors)
        images_w_struct = np.concatenate([images_w_struct, image_flat], axis=1)        
        images_h_struct.append(images_w_struct)
        
    images = np.concatenate(images_h, axis=0)
    scipy.misc.imsave(save_path, images)
    
    images_struct = np.concatenate(images_h_struct, axis=0)
    scipy.misc.imsave(save_path_struct, images_struct)
    
def print_conf_mat(enc, dp, figure_dir, batch_size, default_gpu_id):


    train_or_test_groups = ['test', 'train']

    cf_out = dict()

    for train_or_test in train_or_test_groups:
        save_path = figure_dir + os.sep + 'conf_mat_' + train_or_test + '.csv'
        
        
        y_hat = list()
        y = list()

        ndat = dp.get_n_dat(train_or_test)

        intervals = np.arange(0, ndat, batch_size)

        for i in tqdm(intervals):
            batch_range = i + batch_size
            if batch_range > ndat:
                batch_range = ndat


            dat_inds = np.arange(i, batch_range)

            images = Variable(dp.get_images(dat_inds,train_or_test)).cuda(default_gpu_id)
            labels = dp.get_classes(dat_inds,train_or_test).numpy()

            z = enc(images)

            y_hat.append(np.argmax(z[0].data.cpu().numpy(), axis=1))
            y.append(labels)

            del images, labels, z


        y = np.concatenate(y, axis=0)
        y_hat = np.concatenate(y_hat, axis=0)

        cf = confusion_matrix(y, y_hat)
        pd.DataFrame(cf, columns=dp.label_names).to_csv(save_path, index=False)

        cf_out[train_or_test] = cf 
        
    return cf_out

def print_recon(enc, dec, dp, save_path, default_gpu_id, colors=[[1,0,1], [1,1,0], [0,1,1]]):
    
    train_or_test_groups = ['train', 'test'] 

    class_names = dp.label_names
    

    images_train_test = list()
    
    for train_or_test in train_or_test_groups:
        
        ntrain = dp.get_n_dat(train_or_test)
        train_classes = dp.get_classes(np.arange(0, ntrain), train_or_test).numpy()
        
        image_in = list()
        image_out = list()
        
        for i in range(0, len(class_names)):
            inds = np.where(train_classes == i)[0]
#             np.random.shuffle(inds)

            #make the CMY image
            images = dp.get_images([inds[0]], train_or_test)

            image_in_flat = tensor2img(images, colors)
            image_in.append(image_in_flat)
            
            image_recon = dec(enc(Variable(images).cuda(default_gpu_id)))
            image_out_flat = tensor2img(image_recon.data.cpu(), colors)
            image_out.append(image_out_flat)

#             #make the struct-only Yellow image
#             images[:, [0,2], :, :,:] = images[:, [0,2], :, :,:].fill_(0) #not sure why i have to do it this way, but it doesn't work without the assigment

#             images_flat_struct = tensor2img(images, colors)
#             image_list_struct.append(images_flat_struct)
        
        
        image_in = np.concatenate(image_in, axis=1)
        image_out = np.concatenate(image_out, axis=1)
        
        images = np.concatenate([image_in, image_out], axis=0)
        images_train_test.append(images)
        
    images_train_test = np.concatenate(images_train_test, axis=0)
    scipy.misc.imsave(save_path, images_train_test)
    
#     images_struct = np.concatenate(image_list_struct, axis=0)
#     scipy.misc.imsave(save_path_struct, images_struct)


def print_history(logger, save_path, r_or_s):
    field_order = [
                     'decDLoss',
                     'minimaxDecDLoss',
                     'encDLoss',
                     'minimaxEncDLoss',
                     'reconLoss',
                     'classLoss',
                     'refLoss']
    
    
    field_dict = {'reconLoss': '$\mathcal{L}_{X_{' + r_or_s + '}}$',
                  'minimaxEncDLoss': '$\mathcal{L}_{mmEncD^{' + r_or_s + '}}$',
                  'encDLoss': '$\mathcal{L}_{EncD^{' + r_or_s + '}}$',
                  'minimaxDecDLoss': '$\mathcal{L}_{mmDecD^{' + r_or_s + '}}$',
                  'decDLoss': '$\mathcal{L}_{DecD^{' + r_or_s + '}}$',
                  'classLoss': '$\mathcal{L}_{y}$',
                  'refLoss': '$\mathcal{L}_{\hat{x}^{' + r_or_s + '}}$'}
    
    
    plt.figure()

    for i in range(0, len(field_order)):
        field = field_order[i]
        if field in logger.fields:
            plt.plot(logger.log['iter'], logger.log[field], label=field_dict[field])
    
    plt.legend()
#     plt.title('History')
    plt.xlabel('iteration')
    plt.ylabel('loss')
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
def print_latent_plot(embeddings, save_path, r_or_s, dims=[0,1]):
    
    train_or_test_groups = ['train', 'test'] 
    
    plt.figure(num=None, figsize=(4, 4), dpi=120, facecolor='w', edgecolor='k')
    
    handles = list()
    for train_or_test in train_or_test_groups:
        embed_points = embeddings[train_or_test].numpy()
        h = plt.scatter(embed_points[:,dims[0]], embed_points[:,dims[1]], s=1, alpha=0.5)
        
        h = mpatches.Patch(color=h.get_facecolor()[0], label=train_or_test)
        handles.append(h)
        
    plt.xlabel('$z^{' + r_or_s + '}_{' + str(dims[0]+1) + '}$')
    plt.ylabel('$z^{' + r_or_s + '}_{' + str(dims[1]+1) + '}$')    
            
    plt.legend(handles, train_or_test_groups, loc=1)
        
    ax_range = 4
#     plt.axis([-ax_range, ax_range, -ax_range, ax_range])
#     plt.axis('equal')
    
    plt.xlim(-ax_range, ax_range)
    plt.ylim(-ax_range, ax_range)
    plt.gca().set_aspect('equal', adjustable='box')


    plt.savefig(save_path, bbox_inches='tight')
    
    plt.close('all')
    
    pass
    
def print_latent_grid(dec, ref_or_class_num, save_path, default_gpu_id, ndims = 128, nclasses = 10, dims=[0,1], std=1.5, nsamples=5, colors=[[1,0,1], [1,1,0], [0,1,1]]):
    
    z = list()
    
    if ref_or_class_num is not 'ref':
        label = Variable(torch.Tensor(1, nclasses).fill_(-50).cuda(default_gpu_id))
        label.data[0][ref_or_class_num] = 0

        z.append(label)
        
        z.append(Variable(torch.Tensor(1, ndims).fill_(0).cuda(default_gpu_id)))
        


    
    z.append(Variable(torch.Tensor(1, ndims).fill_(0).cuda(default_gpu_id)))
    
    samples = np.linspace(-std, std, nsamples)
    
    images_w = list()
    for i in samples:
        images_h = list()
        for j in samples:
            z[-1].data[0][dims[0]] = i
            z[-1].data[0][dims[1]] = j
    
#             pdb.set_trace()
            im_out = dec(z)
            image_out_flat = tensor2img(im_out.data.cpu(), colors)
            images_h.append(image_out_flat)
            
        images_w.append(np.concatenate(images_h, axis=0))
        
    image_out = np.concatenate(images_w, axis=1)
    
    return image_out
    
def print_image_counts(dp, save_path):
    n_train = dp.get_n_dat('train')
    n_test = dp.get_n_dat('test')
    labels_train = dp.get_classes(np.arange(0,n_train), 'train').numpy()
    labels_test = dp.get_classes(np.arange(0,n_test), 'test').numpy()
    ulabels = np.unique(labels_train)
    
    image_counts = list()
    for label in ulabels:
        label_name = dp.label_names[label]
        n_train_class = len(np.where(labels_train == label)[0])
        n_test_class = len(np.where(labels_test == label)[0])
        
        image_counts.append([label_name, n_train_class+n_test_class, n_train_class, n_test_class])
    
    df = pd.DataFrame(image_counts, columns={'Label name', '#total', '#train', '#test'})
    df.to_csv(save_path, index=False)


In [ ]:
print_image_counts(dp, figure_dir + os.sep + 'label_counts.csv')

In [ ]:
fig1_path = figure_dir + os.sep + 'fig1.png'
fig1_s1_path = figure_dir + os.sep + 'fig1_s1.png'
print_fig1(dp, fig1_path, fig1_s1_path)


In [ ]:
fig2_path = figure_dir + os.sep + 'fig2.png'
fig2_s2_path = figure_dir + os.sep + 'fig2_s2.png'
print_fig2(enc, dec, dp, fig2_path, fig2_s2_path, n_per_row=6, required_test_inds=[898])

In [ ]:
history_ref_path = figure_dir + os.sep + 'ref_history.png'
print_history(logger_ref, history_ref_path, 'r')

history_struct_path = figure_dir + os.sep + 'struct_history.png'
print_history(logger, history_struct_path, 's')



In [ ]:
recon_path = figure_dir + os.sep + 'struct_recon.png'
print_recon(enc, dec, dp, recon_path, gpu_ids[0])


recon_path = figure_dir + os.sep + 'ref_recon.png'
print_recon(enc_ref, dec_ref, dp_ref, recon_path, gpu_ids[0], colors=[[1,0,1], [0,1,1]])

In [3]:
conf_mat_path = figure_dir + os.sep + 'conf_mat'
cf_out  = print_conf_mat(enc, dp, figure_dir, 100, gpu_ids[0])


  0%|          | 0/16 [00:00<?, ?it/s]/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
 19%|█▉        | 3/16 [00:34<02:27, 11.34s/it]

KeyboardInterrupt: 

In [30]:
save_path = figure_dir + os.sep + 'conf_mat_' + 'test' + '.csv'
os.path.exists(save_path)

cm = pd.read_csv(save_path).as_matrix()

eye = np.identity(cm.shape[0]).astype(bool)

print(cm)

n_correct = np.sum(cm[eye])
n = np.sum(cm)

acc = n_correct / n

print('n correct: {0}'.format(n_correct))
print('n total: {0}'.format(n))
print('Accuracy: {0}'.format(acc))

[[ 36   0   4   1   0   0   5   1   0   0   0]
 [  1 330   5   1   2   2   1   0   2   1   0]
 [  3   9 160   1   0   0   1   0   0   0   0]
 [  0   0   0 229   0   0   0   4   0   0   3]
 [  0   0   0   0 104   0   0   0   0   0   0]
 [  0   0   0   0   0 434   0   1   0   2   0]
 [  4   0   0   5   0   0   7   0   0   0   3]
 [  0   0   0   6   0   0   0 156   0   0   0]
 [  0   2   0   0   0   0   0   0 218   1   0]
 [  0   0   0   0   0   0   0   0   1 449   0]
 [  0   0   0   0   0   0   0   0   0   0  17]]
n correct: 2140
n total: 2212
Accuracy: 0.9674502712477396


In [27]:
cm[eye] = 0

np.sum(cm)

123

In [ ]:
[107, np.random.choice(embeddings['test'].shape[1],1)[0]]

In [ ]:
latent_recon_path = figure_dir + os.sep + 'ref_latent.png'
embeddings_path = parent_dir + os.sep + 'ref_model' + os.sep + 'embeddings.pkl'
embeddings = model_utils.load_embeddings(embeddings_path)

print_latent_plot(embeddings, latent_recon_path, 'r', [23,44])


latent_recon_path = figure_dir + os.sep + 'struct_latent.png'
embeddings_path = parent_dir + os.sep + 'struct_model' + os.sep + 'embeddings.pkl'
embeddings = model_utils.load_embeddings(embeddings_path)

print_latent_plot(embeddings, latent_recon_path, 's', [106, 32])


In [ ]:
std = 10
ndims = opt.nRef

nclasses = 10

# z = enc(dp.get_images([0], 'train'))



latent_grid_path = figure_dir + os.sep + 'grid_ref.png'
image_out = print_latent_grid(dec_ref, 'ref', latent_grid_path, gpu_ids[0], std=std, colors=[[1,0,1], [0,1,1]], ndims = ndims, nclasses = nclasses)

scipy.misc.imsave(latent_grid_path, image_out)

image_out_ref = print_latent_grid(dec_ref, 'ref', latent_grid_path, gpu_ids[0], std=0, colors=[[1,0,1], [0,1,1]], ndims = ndims, nclasses = nclasses)

for i in range(0, dp.get_n_classes()):
    print('Printing ' + str(i) + os.sep + str(dp.get_n_classes()))
#     latent_grid_path = figure_dir + os.sep + 'grid_struct_' + dp.label_names[i] + '.png'
#     image_out = print_latent_grid(dec, i, latent_grid_path, gpu_ids[0], std=std, colors=[[1,0,1], [1,1,0], [0,1,1]], ndims = ndims, nclasses = nclasses)
#     scipy.misc.imsave(latent_grid_path, image_out)
    
    latent_grid_path = figure_dir + os.sep + 'grid_struct_' + dp.label_names[i] + '_struct_only.png'
    image_out = print_latent_grid(dec, i, latent_grid_path, gpu_ids[0], std=std, colors=[[0,0,0], [1,1,0], [0,0,0]], ndims = ndims, nclasses = nclasses)
    scipy.misc.imsave(latent_grid_path, image_out)
    
    image_out = image_out + image_out_ref
#     image_out[:,:,2] = image_out_ref[:,:,1]
    
    latent_grid_path = figure_dir + os.sep + 'grid_struct_' + dp.label_names[i] + '.png'
    scipy.misc.imsave(latent_grid_path, image_out)




In [ ]:
z = enc(Variable(dp.get_images([0], 'train')).cuda(0))
print(z)

In [ ]:

print(np.where(dp.data['test']['inds']==9049))


# print(np.where(dp.data['test']['inds']==8824))

# print(np.where(dp.data['test']['inds']==9238))

# print(np.where(dp.data['test']['inds']==9538))

# print(np.where(dp.data['test']['inds']==9695))

In [ ]:
save_path = figure_dir + os.sep + 'tmp.png'
colors=[[1,0,1], [1,1,0], [0,1,1]]

images = Variable(dp.get_images([1], 'train')).cuda(gpu_ids[0])
z = enc(images)
z[1].data[0].fill_(0)
img_out = dec(z)
img_out = img_out.data.cpu()


scipy.misc.imsave(save_path, tensor2img(img_out, colors))

print(z)

In [ ]:
np.arange(0,128,2)

In [ ]:
dp.get_n_dat('train')+dp.get_n_dat('test')

In [ ]:
dp.label_names